

# Лабораторная работа №1: Проведение исследований с алгоритмом KNN



## Задача классификации

## 1. Выбор начальных условий

Для задачи классификации был выбран датасет Heart Failure Prediction Dataset. Основная практическая ценность этого данных заключается в создании прогностических моделей для улучшения диагностики и лечения сердечной недостаточности.

In [ ]:
from functools import partial
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.neighbors import KNeighborsClassifier

cdata= pd.read_csv("heart.csv")

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print(cdata.head())

In [ ]:
   Age Sex ChestPainType  RestingBP  Cholesterol  FastingBS RestingECG  MaxHR ExerciseAngina  Oldpeak ST_Slope  HeartDisease
0   40   M           ATA        140          289          0     Normal    172              N      0.0       Up             0
1   49   F           NAP        160          180          0     Normal    156              N      1.0     Flat             1
2   37   M           ATA        130          283          0         ST     98              N      0.0       Up             0
3   48   F           ASY        138          214          0     Normal    108              Y      1.5     Flat             1
4   54   M           NAP        150          195          0     Normal    122              N      0.0       Up             0

In [346]:
cdata.info()

In [ ]:
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB

В датасете присутствуют категореальные признаки, представленные в виде строк. Чтобы он стал пригоден для обучения модели их нужно преобразовать в набор численных признаков. Для этого воспользуемся one-hot encoding.

In [ ]:
columns_to_encode = ['RestingECG', 'ChestPainType', 'Sex', 'ExerciseAngina', 'ST_Slope']
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_features = encoder.fit_transform(cdata[columns_to_encode])
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(columns_to_encode))
bl_cdata = pd.concat([cdata.drop(columns_to_encode, axis=1).reset_index(drop=True), encoded_df], axis=1)
print(bl_cdata.head())

In [ ]:
   Age  RestingBP  Cholesterol  FastingBS  MaxHR  ...  ExerciseAngina_N  ExerciseAngina_Y  ST_Slope_Down  ST_Slope_Flat  ST_Slope_Up
0   40        140          289          0    172  ...               1.0               0.0            0.0            0.0          1.0
1   49        160          180          0    156  ...               1.0               0.0            0.0            1.0          0.0
2   37        130          283          0     98  ...               1.0               0.0            0.0            0.0          1.0
3   48        138          214          0    108  ...               0.0               1.0            0.0            1.0          0.0
4   54        150          195          0    122  ...               1.0               0.0            0.0            0.0          1.0

# Выбор метрик качества

1. Accuracy измеряет долю правильно классифицированных объектов от общего числа объектов. Она показывает, насколько часто модель выдает верные прогнозы. Это простая и интуитивно понятная метрика, которая дает общее представление о том, насколько хорошо работает модель.
2. Precision измеряет долю истинно-положительных результатов среди всех объектов, которые модель отнесла к положительному классу. Она показывает, насколько можно доверять прогнозам модели, если она предсказала положительный класс.
3. Recall измеряет долю истинно-положительных результатов среди всех объектов, которые на самом деле принадлежат к положительному классу. Она показывает, насколько хорошо модель находит все объекты, которые принадлежат к целевому классу.

## 2. Создание бейзлайна и оценка качества

Разделение на признаки (X) и целевую переменную (y). Далее разделение на обучающую и тестовую выборки

In [ ]:
model = KNeighborsClassifier(n_neighbors=best_k)
model.fit(X_train_scaled, y_train)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Поиск оптимального k для KNN. Далее создание и обучение модели

In [ ]:
param_grid = {'n_neighbors': range(1, 21)}
knn = KNeighborsClassifier()
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_scaled, y_train)

best_k = grid_search.best_params_['n_neighbors']
print(f"Optimal k for KNN: {best_k}")


model = KNeighborsClassifier(n_neighbors=best_k)
model.fit(X_train_scaled, y_train)

Предсказание на тестовой выборке и оценка качества

In [ ]:
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)

print(f"Метрики для классификации (KNN):")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")

In [ ]:
Метрики для классификации (KNN):
  Accuracy: 0.8641
  Precision: 0.8664
  Recall: 0.8641

Точность для встроенных моделей sklearn оказалась очень хорошей. Попробуем улучшить бейзлайн:

## 3. Улучшение бейзлайна

Гипотеза: Добавление Полиномиальных Признаков может улучшить модель. Взаимодействия между признаками могут иметь важное значение для предсказания. Добавление полиномиальных признаков (например, квадратов и попарных произведений исходных признаков) может помочь модели уловить нелинейные зависимости.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

def train_and_evaluate(X_train, y_train, X_test, y_test, k):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    return accuracy, precision, recall


accuracy_base, precision_base, recall_base = train_and_evaluate(X_train_scaled, y_train, X_test_scaled, y_test, best_k)
print(f"Метрики без полиномиальных признаков:")
print(f"  Accuracy: {accuracy_base:.4f}")
print(f"  Precision: {precision_base:.4f}")
print(f"  Recall: {recall_base:.4f}")

for degree in range(2, 4): 
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_train_poly = poly.fit_transform(X_train_scaled)
    X_test_poly = poly.transform(X_test_scaled)

    accuracy_poly, precision_poly, recall_poly = train_and_evaluate(X_train_poly, y_train, X_test_poly, y_test, best_k)
    print(f"Метрики с полиномиальными признаками (степень {degree}):")
    print(f"  Accuracy: {accuracy_poly:.4f}")
    print(f"  Precision: {precision_poly:.4f}")
    print(f"  Recall: {recall_poly:.4f}")


In [ ]:
Метрики с полиномиальными признаками (степень 2):
  Accuracy: 0.8696
  Precision: 0.8702
  Recall: 0.8696

Использование полиномиальных признаков степени 2 не сильно, но улучшило результаты. Помимо этого в коде используется поиск оптимального k для KNN с weights='uniform', что увеличивает эффективность модели. Также была провереная гипотеза, что отбор наиболее информативных признаков может уменьшить шум в данных, улучшить обобщающую способность модели и, возможно, уменьшить вычислительные затраты. Однако он не продемонстрировал никакой эффективности. 

## 4. Имплементация алгоритма машинного обучения 

Напишем собственную реализацию алгоритма KNN:

In [ ]:
class KNN:
    def __init__(self, k=3):
        self.k = k
        self.X_train = None
        self.y_train = None

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def _euclidean_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2)**2))

    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return np.array(y_pred)

    def _predict(self, x):
        distances = [self._euclidean_distance(x, x_train) for x_train in self.X_train]
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        most_common = Counter(k_nearest_labels).most_common(1)
        return most_common[0][0]

Обучаем имплементированные модели и оцением их качество

In [ ]:
X = bl_cdata.drop("HeartDisease", axis=1).values
y = bl_cdata["HeartDisease"].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


knn_model = KNN(k=5) 
knn_model.fit(X_train_scaled, y_train)


y_pred = knn_model.predict(X_test_scaled)


accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)

print(f"Метрики для KNN (имплементированный):")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")

In [ ]:
Метрики для KNN (имплементированный):
  Accuracy: 0.8641
  Precision: 0.8653
  Recall: 0.8641

Как видно имплементрованный KNN почти так же хорошо. Добавим техники из улучшенного бейзлайна 

In [ ]:
X = bl_cdata.drop("HeartDisease", axis=1)
y = bl_cdata["HeartDisease"]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)


best_k = None
best_accuracy = 0
for k in range(1, 21):
  knn_model = KNN(k=k)
  knn_model.fit(X_train_poly, y_train.values)

  y_pred = knn_model.predict(X_test_poly)
  accuracy = accuracy_score(y_test, y_pred)

  if accuracy > best_accuracy:
      best_accuracy = accuracy
      best_k = k

print(f"Optimal k for KNN: {best_k}")

knn_model = KNN(k=best_k)
knn_model.fit(X_train_poly, y_train.values)
y_pred = knn_model.predict(X_test_poly)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)

print(f"Метрики для KNN (имплементированный, с полиномиальными признаками 2 степени и оптимальным k):")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")


In [ ]:
Метрики для KNN (имплементированный, с полиномиальными признаками 2 степени и оптимальным k):
  Accuracy: 0.8696
  Precision: 0.8699
  Recall: 0.8696

После улучшения, бейзлайн теперь выдаёт результаты, которые практически не отличаются от результатов библиотечной реализации.

## Задача регрессии

Для этой задачи был выбран датасет Depression Student Dataset. Датасет учащихся с депрессией может быть очень полезным для исследований и разработки решений в области психического здоровья, образования и социального благополучия. 

In [ ]:
from functools import partial
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.neighbors import KNeighborsClassifier

rdata = pd.read_csv("DepressionStudentDataset.csv")

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print(rdata.head())

In [ ]:
   Gender  Age  Academic Pressure  Study Satisfaction     Sleep Duration Dietary Habits Have you ever had suicidal thoughts ?  Study Hours  Financial Stress Family History of Mental Illness Depression
0    Male   28                2.0                 4.0          7-8 hours       Moderate                                   Yes            9                 2                              Yes         No
1    Male   28                4.0                 5.0          5-6 hours        Healthy                                   Yes            7                 1                              Yes         No
2    Male   25                1.0                 3.0          5-6 hours      Unhealthy                                   Yes           10                 4                               No        Yes
3    Male   23                1.0                 4.0  More than 8 hours      Unhealthy                                   Yes            7                 2                              Yes         No
4  Female   31                1.0                 5.0  More than 8 hours        Healthy                                   Yes            4                 2                              Yes         No

In [ ]:
rdata.info()

In [ ]:
RangeIndex: 502 entries, 0 to 501
Data columns (total 11 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Gender                                 502 non-null    object 
 1   Age                                    502 non-null    int64  
 2   Academic Pressure                      502 non-null    float64
 3   Study Satisfaction                     502 non-null    float64
 4   Sleep Duration                         502 non-null    object 
 5   Dietary Habits                         502 non-null    object 
 6   Have you ever had suicidal thoughts ?  502 non-null    object 
 7   Study Hours                            502 non-null    int64  
 8   Financial Stress                       502 non-null    int64  
 9   Family History of Mental Illness       502 non-null    object 
 10  Depression                             502 non-null    object 
dtypes: float64(2), int64(3), object(6)
memory usage: 43.3+ KB

В датасете присутствуют категореальные признаки, представленные в виде строк. Чтобы он стал пригоден для обучения модели их нужно преобразовать в набор численных признаков. Для этого воспользуемся one-hot encoding.